I came across a YT video about 'Make your Agents teachable!'.

I thought I had missed something...

In reality, it was extracting data from a response and then storing it in a DB so that it could be retrieved as needed in the future.

Again, traditional Python not some new programming structure.


In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from pprint import pprint

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins:{openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins:sk-proj-TVyeca...


In [3]:
client = OpenAI()
MODEL = "gpt-4o-mini"

Here we instruct the Agent to extract data from the response and also use some XML type tags as part of our prompt. Many LLMs use this feature in their responses.


I am going to use some different syntax just to show that we can create our own output formats. We can use CAPS and Markdown styling to emphasise important points.


In [4]:
system_message = """
You are a teachability agent. You examine a conversation listed between <conv> and </conv> and output a list of pertinent facts,
as well as a concise summary. 
The OUTPUT FORMAT *must have* in the following JSON FORMAT:

{
    "summary": <SUMMARY>,
    "number_of_facts": <NUMBER_OF_FACTS>,
    "facts": [<FACTS>]
}

A fact is a dictionary with the following keys: "fact" and "catgerory".

Here are some examples of facts:

{"fact": "Charles is a vegan and won't eat any meat.", "category": "personal"}
{"fact": "Charles works in Brighton", "category": "work"}
{"fact": "They have four dogs", "category": "pets"}

## NOT A FACT
This is not a fact as it does not refer to a person:
"London is a city and the capital of England"

## NUMBER_OF_FACTS
<NUMBER_OF_FACTS> stores the number of facts in the "facts" list
*Be as specific as you can about the categories*
"""

In [5]:
data = """Peter is a vegan and won't eat any meat. He has been a vegan for over five years when he met his current wife Angela. They have two dogs, Roxy and Petra, and they both eat meat.

They both work in London but live in Brighton near Seven Dials.

They travel a lot and have visited the following countries in the last year - Italy, France and Germany.

They have an active YouTube channel where they post videos about their travels.

They have a cat named Marmalade.

Brighton is based on the South Coast of England and is known for its beaches and nightlife."""

In [6]:
system_message += "<conv>" + data + "</conv>"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": ""},
]

In [8]:
response = client.chat.completions.create(model=MODEL, messages=prompts, temperature=0)
res = response.choices[0].message.content.replace("\n", "")

In [9]:
output = json.loads(res)

Having extracted information, we can store this in 'short term memory' - app level cache - or long term memory - stored in a DB for example
and retrieved as needed and added to the SYSTEM MESSAGE.


In [10]:
pprint(output)

{'facts': [{'category': 'personal',
            'fact': "Peter is a vegan and won't eat any meat."},
           {'category': 'personal',
            'fact': 'Peter has been a vegan for over five years.'},
           {'category': 'personal', 'fact': 'Peter is married to Angela.'},
           {'category': 'pets',
            'fact': 'They have two dogs named Roxy and Petra.'},
           {'category': 'pets', 'fact': 'They have a cat named Marmalade.'},
           {'category': 'work', 'fact': 'They both work in London.'},
           {'category': 'location',
            'fact': 'They live in Brighton near Seven Dials.'},
           {'category': 'travel',
            'fact': 'They have traveled to Italy, France, and Germany in the '
                    'last year.'}],
 'number_of_facts': 8,
 'summary': 'Peter is a vegan who has been so for over five years. He lives in '
            'Brighton with his wife Angela, and they have two dogs and a cat. '
            'They both work in London and 

In [11]:
pprint(output["summary"])
pprint(output["facts"])

('Peter is a vegan who has been so for over five years. He lives in Brighton '
 'with his wife Angela, and they have two dogs and a cat. They both work in '
 'London and have traveled to several countries in the past year, while also '
 'maintaining an active YouTube channel about their travels.')
[{'category': 'personal', 'fact': "Peter is a vegan and won't eat any meat."},
 {'category': 'personal',
  'fact': 'Peter has been a vegan for over five years.'},
 {'category': 'personal', 'fact': 'Peter is married to Angela.'},
 {'category': 'pets', 'fact': 'They have two dogs named Roxy and Petra.'},
 {'category': 'pets', 'fact': 'They have a cat named Marmalade.'},
 {'category': 'work', 'fact': 'They both work in London.'},
 {'category': 'location', 'fact': 'They live in Brighton near Seven Dials.'},
 {'category': 'travel',
  'fact': 'They have traveled to Italy, France, and Germany in the last year.'}]
